In [1]:
import sys

sys.path.append('../../other_repos/')
sys.path.append('../../helperToolz/')
from helper import *
from helpsters import * 

# sys.path.append('/home/potzschf/repos/')
# from helperToolz.helpsters import *


####################################################### Prepare

# make vrts from force outputs for easier processing
year = 2023
reduced_files = reduce_forceTSA_output_to_validmonths(f'/data/Aldhani/eoagritwin/force/output/BRANDENBURG/{year}/', 3, 8)
# ordered_files = force_order_Colors_for_VRT(reduced_files, ['BLU', 'GRN', 'RED', 'BNR'], [f'MONTH-{d:02d}' for d in range(3,9,1)])
# force_to_vrt(reduced_files, ordered_files, f'/data/Aldhani/eoagritwin/fields/Auxiliary/vrt/{year}/', True, bandnames=['BLU', 'GRN', 'RED', 'BNR'])


INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [ ]:
/home/fields/

TypeError: dirname() missing 1 required positional argument: 'p'

In [ ]:
list_of_forcefiles=reduced_files
ordered_forcetiles=ordered_files,
vrt_out_path=f'/data/Aldhani/eoagritwin/fields/Auxiliary/vrt/{year}/'
pyramids=True
bandnames=['BLU', 'GRN', 'RED', 'BNR']

    
# tiles = list(set([file.split('output/')[-1].split('/')[1].split('/')[0] for file in list_of_forcefiles]))
force_folder_name = getFORCExyRangeName(get_forcetiles_range(list_of_forcefiles))
if not vrt_out_path.endswith('/'):
    vrt_out_path = vrt_out_path + '/'
outDir = f'{vrt_out_path}{force_folder_name}/'
if not os.path.exists(outDir):
    os.makedirs(outDir)
    print(outDir)
    vrts = []

In [ ]:
for i in range(len(ordered_forcetiles)):
        vrt_name = f'{outDir}{force_folder_name}_{str(i)}.vrt'
        vrt = gdal.BuildVRT(vrt_name, ordered_forcetiles[i], separate = False)
        vrt = None

        # make paths in vrts relative
        convertVRTpathsTOrelative(vrt_name)
        vrts.append(vrt_name)

# set optionally bandnames    
if bandnames:
    for idz, bname in enumerate(bandnames): 
        print(f'{outDir}{force_folder_name}_{str(idz)}.vrt')
        vrt = gdal.Open(f'{outDir}{force_folder_name}_{str(idz)}.vrt', gdal.GA_Update)  # VRT must be writable
        band = vrt.GetRasterBand(1)
        band.SetDescription(bname)
        vrt = None
print('single vrts created')
    
nums = [int(vrt.split('_')[-1].split('.')[0]) for vrt in vrts]
vrts_sorted = sortListwithOtherlist(nums, vrts)[-1]
print('paths in vrts made relative')

vrt = gdal.BuildVRT(f'{outDir}{force_folder_name}_Cube.vrt', vrts_sorted, separate = True)
vrt = None
if bandnames:
    # set vrt band names
    vrt = gdal.Open(f'{outDir}{force_folder_name}_Cube.vrt', gdal.GA_Update)  # VRT must be writable
    for idz, bname in enumerate(bandnames): 
        band = vrt.GetRasterBand(1+idz)
        band.SetDescription(bname)
    vrt = None
# convertVRTpathsTOrelative(f'{outDir}{force_folder_name}_Cube.vrt')
print('overlord vrt created')
if pyramids:
    # build pyramids
    vrtPyramids(f'{outDir}{force_folder_name}_Cube.vrt')
    print('VRT created with pyramids')



In [ ]:
# load vrts into npdstack
dat = loadVRTintoNumpyAI4(f'/data/fields/Auxiliary/vrt/{year}/{getFORCExyRangeName(get_forcetiles_range(reduced_files))}')

# set tiling scheme and chip size on which prediction will be undertaken
chipsize = 128*1 # 5 is the maximum with GPU in basement
overlap  = 20

row_col_ind = get_row_col_indices(chipsize, overlap, dat.shape[2:][0], dat.shape[2:][1])

####################################################### Predict

predicted_chips_list = predict_on_GPU('/data/fields/output/model_state_All_but_LU_transformed_42.pth', row_col_ind, dat)

In [ ]:
'/data/fields/output/rocks_db/AI4_RGB_NDVI_exclude_false.db'

In [ ]:
from math import ceil as mceil

img_size = 128
## create metadata file
metadata = {
    'inputs': {
        'inputs_shape': (5, 6, img_size, img_size),  
        'inputs_dtype': np.float32     
    },
    'labels': {
        'labels_shape': (4, img_size, img_size),          
        'labels_dtype': np.float32
    }
}

## define function to load imgs and labs
def names2array_function(names):

    variables2use=['B2','B3','B4','B8','NDVI']

    image_path, label_path = names
    # load image
    img = xr.open_dataset(image_path)
    image = np.concatenate([img[var].values[None] for var in variables2use],0)
    
    # load label
    ds = xr.open_dataset(label_path)
    label = np.asarray(ds['band_data'].values)
    label[np.isnan(label)] = 0
    return [image, label] 

def names2array_function16(names):

    variables2use=['B2','B3','B4','B8']#,'NDVI']

    image_path, label_path = names
    # load image
    img = xr.open_dataset(image_path)
    image = np.concatenate([img[var].values[None] for var in variables2use],0)
    
    # load label
    ds = xr.open_dataset(label_path)
    label = np.asarray(ds['band_data'].values)
    label[np.isnan(label)] = 0
    return [image.astype(np.float16), label.astype(np.float16)]  

In [ ]:
## create list of images and labels

folders = ['LU', 'AT', 'ES', 'FR', 'NL', 'SE', 'SI']
imgs = [getFilelist('/home/ai4boundaries/sentinel2/images/' + folder, '.nc') for folder in folders]
imgs = [img for list in imgs for img in list]
labs = [getFilelist('/home/ai4boundaries/sentinel2/masks/' + folder, '.tif') for folder in folders]
labs = [lab for list in labs for lab in list]
print(len(imgs), len(labs))
imgs.sort()
labs.sort()

## exclude completely empty images
exclude = False
if exclude == True:
    aa = checkemptyNC(labs)
    # exclude images with empty labels and their labels
    labs = [lab for i, lab in enumerate(labs) if i not in aa]
    imgs = [img for i, img in enumerate(imgs) if i not in aa]

print(len(imgs), len(labs))


img_lab_paths = []
for i in range(len(imgs)):
    img_lab_paths.append((imgs[i], labs[i]))

In [ ]:
from sklearn.linear_model import LinearRegression
from skimage import measure
from joblib import Parallel, delayed

import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *


#########################  parameter settings
# load the predictions and labels

reference  = '/data/fields/IACS/4_Crop_mask/GSA-DE_BRB-2019_cropMask_lines_touch_true_lines_touch_true_linecrop.tif'
    
predictions =  '/data/fields/output/predictions/FORCE/BRANDENBURG/vrt/256_20_chips.vrt' # predictions straight from GPU 
#reference =  '/data/fields/IACS/4_Crop_mask/GSA-DE_BRB-2019_cropMask_lines_touch_true_lines_touch_true_linecrop.tif' # mask from IACS
result_dir = '/data/fields/Auxiliary/grid_search/Brandenburg/' + predictions.split('/')[-1].split('.')[0] + '_masked_with_and_preds_are_' + reference.split('/')[-1].split('.')[0]
sub = predictions.split('/')[-1].split('.')[0] + '_masked_with_and_preds_are_' + reference.split('/')[-1].split('.')[0]
folder_path = f'/data/fields/output/predictions/FORCE/BRANDENBURG/auxiliary/{sub}/'
vrt_for_folder_path = folder_path + 'vrt/'
os.makedirs(folder_path, exist_ok=True)
os.makedirs(vrt_for_folder_path, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)
# set the number by which rows and cols will be divided --> determines the number of tiles // also set border limit (dont sample fields too close to tile borders) and sample size
slicer = 10
border_limit = 5
sample_size  = 10000
# set the number of cores for parallel processing and set seed
ncores = 20
np.random.seed(42)
make_tifs_from_intermediate_step = True
######### prepare job-list


# create lists that will be passed on to the joblist
tile_list = []
extent_true_list = []
extent_pred_list = []
boundary_pred_list = []
result_dir_list = []
row_col_start = []


# tile predictions in prds --> total extent encompasses 90 Force Tiles (+ a few rows and cols that will be neglected as they are outside of study area)
pred_ds = gdal.Open(predictions)
rows, cols = pred_ds.RasterYSize, pred_ds.RasterXSize

row_start = [i for i in range(0, rows, math.floor(rows/slicer))]
row_end = [i for i in range (math.floor(rows/slicer), rows, math.floor(rows/slicer))]
row_start = row_start[:len(row_end)] 

col_start = [i for i in range(0, cols, math.floor(cols/slicer))]
col_end = [i for i in range (math.floor(cols/slicer), cols, math.floor(cols/slicer))]
col_start = col_start[:len(col_end)] 

In [ ]:
# load IACS reference mask and label it 
ref_ds = gdal.Open(reference)
extent_true = ref_ds.GetRasterBand(1).ReadAsArray()
binary_true = extent_true > 0
instances_true = measure.label(binary_true, background=0, connectivity=1)

# sample fields
# build a mask to exclude fields that are in border_limit to tile borders
power_mask = np.zeros(instances_true.shape)
for i in range(len(row_end)):
    for j in range(len(col_end)):
            power_mask[row_start[i]:row_start[i] + border_limit, :] = 1
            power_mask[:, col_start[j]:col_start[j] + border_limit] = 1
            power_mask[row_end[-1] - border_limit:power_mask.shape[0], :] = 1
            power_mask[:, col_end[-1] - border_limit:power_mask.shape[1]] = 1

# get IDs from labelled reference
IDs_to_skip = np.unique(instances_true[power_mask==1])

# get distribution of field sizes after segmentation
unique_IDs, counts = np.unique(instances_true, return_counts=True)

# exlcude fields that are too close to tile borders
mask = ~np.isin(unique_IDs, IDs_to_skip)
unique_IDs = unique_IDs[mask]
counts = counts[mask]

# exlude 0 (background) and 1 (super-small fields) from sample
mask = (unique_IDs != 0) & (counts > 2)
unique_IDs = unique_IDs[mask]
counts = counts[mask]


# get deciles and draw equally from them
deciles = [perc for perc in range(10,100,10)]
deciles_values = np.percentile(counts, deciles)
decs = [0] + deciles_values.tolist() + [np.max(counts)]
bin_ids = []
for ind in range(len(decs) -1):
    # get the unique_IDS of those fields, whose count (size) is within bin
    bin_ids.append(np.random.choice(unique_IDs[(counts > decs[ind]) & (counts <= decs[ind + 1])], int(sample_size/10), replace=False))

mask = np.isin(instances_true, np.concatenate(bin_ids))
# set everything to 0 except samples
instances_true[~mask] = 0




In [ ]:
i=0
j=6
# read in vrt in tiles

    
######### fill the lists with tiled data


#subset the prediction of fields read-in
extent_pred = pred_ds.GetRasterBand(1).ReadAsArray(col_start[j], row_start[i], col_end[j] - col_start[j], row_end[i] - row_start[i]) # goes into InstSegm --> image of crop probability 
# mask extend_pred with reference
extent_pred_masked = extent_pred * extent_true[row_start[i]:row_end[i], col_start[j]:col_end[j]]


# check if tile contains a sample of reference/label data
extent_true_label = instances_true[row_start[i]:row_end[i], col_start[j]:col_end[j]]

extent_true_list.append(extent_true_label)
extent_pred_list.append(extent_pred_masked)

# make identifier for tile for csv
tile_list.append(f'{str(i)}_{str(j)}')
# load predicted boundary prob subset // goes into InstSegm --> image of boundary probability
boundary_pred_list.append(pred_ds.GetRasterBand(2).ReadAsArray(col_start[j], row_start[i], col_end[j] - col_start[j], row_end[i] - row_start[i])) 
# output folder
result_dir_list.append(result_dir)
row_col_start.append(str(row_start[i]) + '_' + str(col_start[j]))

jobs = [[tile_list[i], row_col_start[i] ,extent_true_list[i], extent_pred_list[i], boundary_pred_list[i], result_dir_list[i],  pred_ds.GetGeoTransform(), pred_ds.GetProjection(), folder_path, border_limit]  for i in range(len(result_dir_list))]


In [ ]:
ind = 0

row_col_start = jobs[ind][1]
extent_true = jobs[ind][2]
extent_pred = jobs[ind][3]
boundary_pred = jobs[ind][4]
t_ext=0.2
t_bound=0.2
border_limit=border_limit = jobs[ind][6]

In [ ]:
row_start = int(row_col_start.split('_')[0])
col_start = int(row_col_start.split('_')[1])
# get predicted instance segmentation
instances_pred = InstSegm(extent_pred, boundary_pred, t_ext=t_ext, t_bound=t_bound)
instances_pred = measure.label(instances_pred, background=-1) 
instances_true = extent_true
field_values = np.unique(instances_true)




In [ ]:
best_IoUs = []
field_IDs = []
field_sizes = []
centroid_rows = []
centroid_cols = []
centroid_IoUS = []
centroid_IDs = []
intersectL  = []


In [ ]:
# for field_value in field_values:
#     if field_value == 0:
#         continue # move on to next value

field_value = field_values[1]
this_field = instances_true == field_value
# # check if field is close to border and throw away if too close
# if TooCloseToBorder(this_field, border_limit):
#     continue

# calculate centroid
this_field_centroid = np.mean(np.column_stack(np.where(this_field)),axis=0).astype(int)

# fill lists with info
centroid_rows.append(this_field_centroid[0])
centroid_cols.append(this_field_centroid[1])
field_IDs.append(field_value)
field_sizes.append(np.sum(this_field))


In [ ]:
print(field_values)

In [ ]:
this_field_centroid

In [ ]:

# find predicted fields that intersect with true field
intersecting_fields = this_field * instances_pred
intersect_values = np.unique(intersecting_fields)

# compute IoU for each intersecting field
field_IoUs = []
center_IoU = 0
for intersect_value in intersect_values:
    if intersect_value == 0:
        field_IoUs.append(0)
        continue # move on to next value
    
    pred_field = instances_pred == intersect_value
    union = this_field + pred_field > 0
    intersection = (this_field * pred_field) > 0
    IoU = np.sum(intersection) / np.sum(union)
    field_IoUs.append(IoU)
    # check for centroid condition
    if instances_pred[this_field_centroid[0], this_field_centroid[1]] == intersect_value:
        center_IoU = IoU
        centroid_IDs.append(field_value)

# take maximum IoU - this is the IoU for this true field
if len(field_IoUs) != 0:
    best_IoUs.append(np.max(field_IoUs))
    # fill centroid list
    centroid_IoUS.append(center_IoU)
    max_index = np.argmax(field_IoUs)
    intersectL.append(intersect_values[max_index])

else:
    best_IoUs.append(0)
    # fill centroid list
    centroid_IoUS.append(0)


# Create mask of intersecting fields with best IoUs
intersect_mask = np.isin(instances_pred, intersectL)
filtered_instances_pred = instances_pred * intersect_mask
# centroids
for r,c, cid in zip(centroid_rows, centroid_cols, centroid_IDs):
filtered_instances_pred[r, c] = cid

In [ ]:
intersecting_fields.shape